In [1]:
from dateutil.parser import parse
import csv
import pickle
import json
import time
import random 
from urllib.request import urlopen
from urllib.parse   import quote
from datetime import datetime, timedelta
import re  
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np  
import os
from selenium import webdriver 
from random import randint
from urllib.request import Request
import requests
import random
import sqlite3
from IPython.display import display #이거 써봐라.. 
pd.options.display.max_columns = None
pd.options.display.max_rows = 300

In [2]:
#시작

In [224]:
from xlrd import open_workbook  #정규학기 불러오기  엑셀 -> df
workbookName = r"D:\해달\knu_major.xlsx" #엑셀파일 불러오기 
sheet = open_workbook(workbookName).sheets()[0] #시트 불러오기 

In [225]:
flag = 0 #엑셀 파일 시트에서 데이터프레임으로 만드는 부분 
out_list = []
개설대학 = ''

for rowidx in range(sheet.nrows):
    
    empty_dict = {}
    
    row = sheet.row(rowidx)
    key = row[3].value    #맨 왼쪽에서 4번째 줄을 구분자로 활용, 숫자가 들어가 있으면 해당 로우 값을 다 저장, 직전의 개설대학 값과 함께
     
    if row[5].value == '수  업  시  간  표':
        flag = 1
        continue
        
    if flag == 1:
        개설대학 = row[1].value
        flag = 0
        continue
        
    try:
        num = int(key[-1])
    except:
        continue
        
    empty_dict['학년'] = row[1].value
    empty_dict['구분'] = row[2].value
    empty_dict['교과번호-분반'] = row[3].value
    empty_dict['교과목명'] = row[4].value
    empty_dict['학점'] = row[6].value
    empty_dict['강의'] = row[7].value
    empty_dict['실습'] = row[8].value
    empty_dict['담당교수'] = row[9].value
    empty_dict['강의시간(요일,교시)'] = row[10].value
    empty_dict['강의실'] = row[13].value
    empty_dict['비고'] = row[14].value
    empty_dict['개설대학'] = 개설대학
    out_list.append(empty_dict) #추가되는 줄, 가장 최근의 개설대학 값을 바탕으로 

In [226]:

def elim_bar(x): #텍스트에서 바 없애는 함수 
    return x.replace('-','')

ex_df = pd.DataFrame(out_list) #데이터 프레임으로 변형
ex_df['교과번호-분반'] = ex_df['교과번호-분반'].map(elim_bar) #학기 
column_list = ['교과번호-분반', '학점', '강의', '실습', '교과목명', '구분', '수강정원', '강의시간(요일,교시)', '강의실',
       '개설대학', '개설\n학과', '개설\n전공', '담당교수', '팀티칭\n여부', '스마트러닝\n강좌 여부', '직급',
       '교수소속\n대학', '교수소속\n학과', '강의\n언어', '교과목명\n(영문)','학년', '비고']
ex_df = ex_df.loc[:,column_list]

In [79]:
#계절학기 불러오기 엑셀 -> df

In [229]:
sw_df = pd.read_excel (r'D:\해달\2019겨울계절수업 개설강좌 수업시간표.xls', header = [2]) #for an earlier version of Excel, you may need to use the file extension of 'xls'

In [230]:
sw_df.columns = ['교과번호-분반', '학점', '강의', '실습', '교과목명', '구분', '수강정원', '강의시간(요일,교시)', '강의실',
       '개설대학', '개설\n학과', '개설\n전공', '담당교수', '팀티칭\n여부', '스마트러닝\n강좌 여부', '직급',
       '교수소속\n대학', '교수소속\n학과', '강의\n언어', '교과목명\n(영문)', '비고'] #학년칼럼이 없어서추가. 
sw_df= sw_df.loc[:,column_list]
sw_df['교과번호-분반'] = sw_df['교과번호-분반'].map(elim_bar)

In [232]:
import urllib.request           # 웹크롤링 모듈
import bs4                      # 웹크롤링 모듈
import time

In [233]:
def Current_enrollment_num(code): # 실시간 수강신청 인원
    code = code.upper()        #과목코드의 영문부분을 모두 대문자로 표기

    ## URL에 들어갈 정보
    # '학기(semester) ex.20192',
    # '과목분반정보(subClass) ex. 001',
    # '과목정보(maincode) ex. ENGR211,
    # '모든코드(code) ex. ENGR211001,

    ####################'학기(Year_semester)'#########################
    ## 월 추출(정수)
    Month_int = int(time.strftime("%m", time.localtime(time.time())))

    ## 년 추출(문자)
    Year_int = str(time.strftime("%Y", time.localtime(time.time())))

    ##학기 추출
    if Month_int <= 3:  # 1, 2, 3월이면
        semester = "1"  # 1학기 수강신청
    elif 4 <= Month_int <= 6:  # 4, 5, 6월이면
        semester = "S"  # 여름학기 수강신청
    elif 7 <= Month_int <= 9:  # 7, 8, 9월이면
        semester = "2"  # 2학기 수강신청
    elif 10 <= Month_int <= 12:  # 10, 11, 12월이면
        semester = "W"  # 겨울학기 수강신청
    Year_semester = Year_int + semester  # ex. Year_semester는 "20192"

    subClass = code[7:]     # '과목분반정보(subClass) ex. 001'
    maincode = code[:7]     # '과목분반정보(subClass) ex. ENGR211'

    url ="http://my.knu.ac.kr/stpo/stpo/cour/lectReqCntEnq/list.action?&lectReqCntEnq.search_open_yr_trm="+Year_semester+"&lectReqCntEnq.search_sub_class_cde="+subClass+"&lectReqCntEnq.search_subj_cde="+maincode+"&searchValue="+code
    html = urllib.request.urlopen(url)      # html 코드 받기
    bsObj = bs4.BeautifulSoup(html, "html.parser")  #bs4에 html 코드 넣기

    # 현 과목코드에 대한 정보가 없을시에 0값을 도출합니다
    try:
        current_num = int(bsObj.find("td", {"class": "lect_req_cnt"}).text)     # 실시간 정원
        limit_num = int(bsObj.find("td", {"class": "lect_quota"}).text)         # 수용정원
    except:
        return [0, 0]

    return [limit_num, current_num]     # [수용정원, 실시간 정원]

In [ ]:
def Get_same_lec(code,df):  #code 입력 과목과 같은 과목 다른 시간
    return_df = df.loc[df.loc[:,'교과번호-분반'].str.contains(code[:7]),:]
    return_df.loc[:,'수용정원'] = 'no'
    return_df.loc[:,'실시간정원'] = 'no'
    for index, row in return_df.iterrows():
        return_list = Current_enrollment_num(row['교과번호-분반'])
        print(return_list)
        return_df.loc[index,'수용정원'] = return_list[0]
        return_df.loc[index,'실시간정원'] = return_list[1]
    #return_df['잔여정원'] = return_df['수용정원'].map(int) - return_df['실시간정원'].map(int)
    return return_df


def Get_same_time(code,df): #code 입력과목과 다른 과목 같은 시간
    for index, value in df.loc[df['교과번호-분반'] == code,'강의시간(요일,교시)'].iteritems(): #더 좋은 방법이 있다면. 이렇게. 
        return_df = df.loc[df['강의시간(요일,교시)'] == value,:]
    for index, row in return_df.iterrows():
        return_list = Current_enrollment_num(row['교과번호-분반'])
        print(return_list)
        return_df.loc[index,'수용정원'] = return_list[0]
        return_df.loc[index,'실시간정원'] = return_list[1]
    return return_df

In [ ]:
#code = 'ENGR211001'
#code = 'CLTR412001'
code = 'CLTR211001'

In [294]:
#Get_same_lec(code,ex_df) #ex_df는 데이터프레임. 정규학기 디비에 대해서 하는 것 

C:\Users\nnqkf\Anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\nnqkf\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


[0, 0]
[65, 57]
[65, 53]
[65, 55]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]


,교과번호-분반,학점,강의,실습,교과목명,구분,수강정원,"강의시간(요일,교시)",강의실,개설대학,개설 학과,개설 전공,담당교수,팀티칭 여부,스마트러닝 강좌 여부,직급,교수소속 대학,교수소속 학과,강의 언어,교과목명 (영문),학년,비고,수용정원,실시간정원
466,CLTR211001,3.0,3,,수학 I,교양,NaN,"수1A,1B,2A 금2B,3A,3B",자연과학대학-318,자연과학대학 수학과,NaN,NaN,임정욱,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*,전학년,0,0
548,CLTR211002,3.0,3,,수학 I,교양,NaN,"화7A,7B,8A 목7A,7B,8A",자연과학대학-B05,자연과학대학 생명과학부 생물학전공,NaN,NaN,이인숙,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,65,57
1433,CLTR211003,3.0,3,,수학 I,교양,NaN,"월2B,3A,3B 금1A,1B,2A",복현회관-201,농업생명과학대학 응용생명과학부,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,신규 강사 담당 예정,65,53
1434,CLTR211004,3.0,3,,수학 I,교양,NaN,"월2B,3A,3B 금1A,1B,2A",복현회관-301,농업생명과학대학 응용생명과학부,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,신규 강사 담당 예정,65,55
1511,CLTR211005,3.0,3,,수학 I,교양,NaN,"화1A,1B,2A 목2B,3A,3B",농대3호관-522,농업생명과학대학 식품공학부,NaN,NaN,이인숙,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,0,0
1512,CLTR211006,3.0,3,,수학 I,교양,NaN,"화1A,1B,2A 목2B,3A,3B",복현회관-301,농업생명과학대학 식품공학부,NaN,NaN,최선미,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,0,0
1611,CLTR211007,3.0,3,,수학 I,교양,NaN,"월1A,1B,2A 수2B,3A,3B",복현회관-201,농업생명과학대학 산림과학.조경학부,NaN,NaN,최선미,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,0,0
2817,CLTR211008,3.0,3,,수학 I,교양,NaN,"수8B,9A,9B",생활과학대학-403,생활과학대학 식품영양학과,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,신규 강사 담당 예정,0,0
3958,CLTR211009,3.0,3,,수학 I,교양,NaN,"월5B,6A,6B 금5B,6A,6B",상주캠퍼스 제9호관-101,과학기술대학 건설방재공학부,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,신규 강사 담당 예정,0,0
3959,CLTR211010,3.0,3,,수학 I,교양,NaN,"월7A,7B,8A 금7A,7B,8A",상주캠퍼스 제9호관-101,과학기술대학 건설방재공학부,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,신규 강사 담당 예정,0,0


In [337]:
test1 = Get_same_lec(code,sw_df) #계절학기 디비에 대해 하는 것 

C:\Users\nnqkf\Anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\nnqkf\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


[0, 0]
[65, 57]
[65, 53]
[65, 55]


In [338]:
#Get_same_time(code,ex_df) 정규학기 디비에 대해 하는 것 

In [339]:
test = Get_same_time(code,sw_df) #계절학기 디비에 대해 하는 것 

[50, 43]
[100, 30]


C:\Users\nnqkf\Anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\nnqkf\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


[70, 23]
[55, 30]
[120, 96]
[120, 85]
[130, 32]
[120, 93]
[140, 111]
[80, 77]
[120, 56]
[70, 68]
[0, 0]
[70, 46]
[30, 31]
[0, 0]
[65, 57]
[65, 33]
[130, 70]
[120, 92]
[80, 50]
[70, 64]
[450, 350]
[70, 55]
[80, 32]
[0, 0]
[90, 84]
[90, 52]
[300, 67]
[45, 42]
[90, 41]
[200, 59]
[80, 80]
[100, 29]
[90, 72]
[90, 82]
[70, 36]
[110, 101]
[90, 63]
[140, 122]
[0, 0]


In [345]:
test = test.applymap(str)
test1 = test1.applymap(str)

In [352]:
text_file = open(r"D:\해달\index.html", "w", encoding ='utf-8')
text_file.write(test.to_html())
text_file.close()

In [353]:
text_file = open(r"D:\해달\index2.html", "w",encoding ='utf-8')
text_file.write(test1.to_html())
text_file.close()

In [ ]:
#휴지통 

In [ ]:
#df['C'] = df.apply(lambda row: row['A'] + row['B'], axis=1)

In [ ]:
for rowidx in range(sheet.nrows):
    row = sheet.row(rowidx)
    if row[5]
        #for colidx, cell in enumerate(row):
            #if cell.value == "particularString" :
             #   print sheet.name
              #  print colidx
               # print rowidx

In [ ]:
db_Link = r"D:\5\5_renew2\youtubenews\0youtube_news.db" 
conn = sqlite3.connect(db_Link)  
cur = conn.cursor()  
vi_count = pd.read_sql_query("select * from vi_count", conn)
conn.close()

conn = sqlite3.connect(db_Link)  
cur = conn.cursor()  
vi_feature = pd.read_sql_query("select * from vi_feature", conn)
conn.close()

conn = sqlite3.connect(db_Link)  
cur = conn.cursor()  
ch_count = pd.read_sql_query("select * from ch_count", conn)
conn.close()

conn = sqlite3.connect(db_Link)  
cur = conn.cursor()  
ch_feature = pd.read_sql_query("select * from ch_feature", conn)
conn.close()